# Create Slice for Fabric Rack
This is the slice creation for the ```Systematic Experimentation (with Ansible)``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib
 

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 3 nodes for the IPv4 Assignment.
### 1.1 Reserve Resources
In this section we will declare the structure of the slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "SystematicExperimentationWithAnsible"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 4
ram = 16
disk = 50

# Create a slice with a Server node and two Client nodes
try:
    #Create Slice
    slice = fablib.new_slice(slice_name)
    
    # Add client node
    node1 = slice.add_node(name='client_0', site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    PortC0 = node1.add_component(model='NIC_Basic', name='CL0-P').get_interfaces()[0]
    # Add client node
    node1 = slice.add_node(name='client_1', site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    PortC1 = node1.add_component(model='NIC_Basic', name='CL1-P').get_interfaces()[0]
    # Add client node
    node1 = slice.add_node(name='client_2', site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    PortC2 = node1.add_component(model='NIC_Basic', name='CL2-P').get_interfaces()[0]
    
    # Add server node
    node2 = slice.add_node(name='server_0', site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    PortS0 = node2.add_component(model='NIC_Basic', name='SV0-P').get_interfaces()[0] 
    # Add server node
    node2 = slice.add_node(name='server_1', site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    PortS1 = node2.add_component(model='NIC_Basic', name='SV1-P').get_interfaces()[0] 

    #add Ansible node
    node3 = slice.add_node(name='ansible', site=site)
    node3.set_capacities(cores=cores, ram=ram, disk=disk)
    node3.set_image(image)
    PortA = node3.add_component(model='NIC_Basic', name='A-P').get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name='Lan-Net', interfaces=[PortC0, PortC1, PortC2, PortS0, PortS1, PortA])

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
This section of the slice creation, once the nodes are active, will assign the network spaces some predefined values that we will use for the Assignment.

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

CL0 = 'client_0'
CL1 = 'client_1'
CL2 = 'client_2'
SV0 = 'server_0'
SV1 = 'server_1'
AN = 'ansible'

LAN = 'Lan-Net'

#CL
CL0_IP = '10.10.0.1' 
CL1_IP = '10.10.0.2' 
CL2_IP = '10.10.0.3' 
#SV
S0_IP = '10.10.1.1'
S1_IP = '10.10.1.2'
#ans
A_IP = '10.10.3.1'

# configure nodes
try:
    CL_0 = slice.get_node(CL0)
    CL_1 = slice.get_node(CL1)
    CL_2 = slice.get_node(CL2)
    SV_0 = slice.get_node(SV0)
    SV_1 = slice.get_node(SV1)
    AN = slice.get_node(AN)
    
    #Client
    C0_iface1= CL_0.get_interface(network_name=LAN) 
    C0_iface1.set_ip(ip=CL0_IP, cidr="16")
    C1_iface1= CL_1.get_interface(network_name=LAN) 
    C1_iface1.set_ip(ip=CL1_IP, cidr="16")
    C2_iface1= CL_2.get_interface(network_name=LAN) 
    C2_iface1.set_ip(ip=CL2_IP, cidr="16")
    
    #Server
    S0_iface2= SV_0.get_interface(network_name=LAN) 
    S0_iface2.set_ip(ip=S0_IP, cidr="16")
    S1_iface2= SV_1.get_interface(network_name=LAN) 
    S1_iface2.set_ip(ip=S1_IP, cidr="16")

    #Ansiblenode
    A_iface2= AN.get_interface(network_name=LAN) 
    A_iface2.set_ip(ip=A_IP, cidr="16")
    
    for node in slice.get_nodes():        
        for iface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev {iface.get_device_name()} up')
        stdout, stderr = node.execute(f'ip a')
        
        node.execute("sudo sysctl -w net.ipv4.ip_forward=1")
except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
In this section of the slice creation, we will add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
import io
import paramiko

try:
    for nd in slice.get_nodes():
        print(f"/*/*/{nd.get_name()}/*/*/")
        nd.upload_file('src/hellogeni-install.tar.gz', 'hellogeni-install.tar.gz')
        stdout, stderr = nd.execute("tar xvfz hellogeni-install.tar.gz")

except Exception as e:
    print(f"Error Uploading tar file: {e}")

try:
    
    for node in slice.get_nodes():
        node.execute('sudo apt-get update')
        node.execute('sudo apt-get upgrade -y')
        node.execute('sudo apt-get install python -y')
        node.execute('sudo apt-get install python-pip -y')
        #install omni requirements
        node.execute('sudo apt-get install swig python-openssl libxmlsec1 xmlsec1 -y\
                     libxmlsec1-openssl libxmlsec1-dev -y')
        #Note: This will set python2 as the default python version on your Ubuntu 20.04 machine.

        node.execute('curl https://bootstrap.pypa.io/pip/2.7/get-pip.py --output get-pip.py')
        node.execute('sudo python2.7 get-pip.py')
        node.execute('sudo apt-get install build-essential swig python-dev libssl-dev -y')
        node.execute('pip2 install M2Crypto==0.30.0')
        node.execute('pip2 install python-dateutil')
        #omni install
        node.execute('wget https://github.com/GENI-NSF/geni-tools/archive/refs/tags/v2.11.tar.gz')

    
        # Extract the tarball
        node.execute('tar -xvzf "v2.11.tar.gz"')
        # Move the extracted directory to the target location
        node.execute('sudo mv "./geni-tools-2.11/" "/usr/local/bin/geni-tools-2.11"')
        node.execute('cd "/usr/local/bin/"')
        node.execute('sudo ln -s geni-tools-2.11/ gcf')
        node.execute('cd "/home/ubuntu/"')

        
        # set PATH so it includes geni software if it exists
        node.upload_file('scripts/install_omni.sh','install_omni.sh')
        node.upload_file('scripts/inventory','inventory')
        node.upload_file('scripts/E2_template','E2_template')
        node.execute('cat install_omni.sh >> .bashrd')
        
    #install Ainsible on anssble node
    stdout, stderr = AN.execute("sudo apt-get install pip -y")
    stdout, stderr = AN.execute("sudo apt install software-properties-common -y" )
    stdout, stderr = AN.execute("sudo add-apt-repository --yes --update ppa:ansible/ansible" )
    stdout, stderr = AN.execute("sudo apt-get install ansible -y" )
    
except Exception as e:
    print(f"Error installing Ansible: {e}")

try:
    key_name = "Ansible_Key"
    #private_key_path = "REPLACE_WITH_PRIVATE_KEY_PATH_FOR_PORTAL_KEYS" {key.get_name()} {key.get_base64()}
    key = paramiko.RSAKey.generate(bits=2048)
    public_key_string = f"{key.get_name()} {key.get_base64()}"
    
    CL_0.add_public_key(sliver_public_key=public_key_string)
    CL_1.add_public_key(sliver_public_key=public_key_string)
    CL_2.add_public_key(sliver_public_key=public_key_string)
    SV_0.add_public_key(sliver_public_key=public_key_string)
    SV_1.add_public_key(sliver_public_key=public_key_string)
    
    x = key.get_name()

    #create a temporary pipe because geting key as a string is not valid
    tmpPipe = io.StringIO()
    key.write_private_key(tmpPipe)
    tmpPipe.seek(0)
    y = tmpPipe.read()

    
    print(f"{x} << echo '{y}'")
    
    stdout, stderr = AN.execute(f"echo '{y}' >> {key_name}")
    stdout, stderr = AN.execute(f"chmod 600 {key_name}")
    
except Exception as e:
    print("Error: sending keys", str(e))

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the Routing IPV4 Notebook. You can either open it on the explorer or click [Here](./HelloAnsible.ipynb) to open the next notebook.